In [1]:
import yfinance as yf
from datetime import datetime
import warnings

# Suprimir avisos específicos
warnings.filterwarnings("ignore", category=FutureWarning, module="yfinance")


In [2]:
# Criar objeto do ticker
ticker_data = yf.Ticker("MSFT")

In [ ]:
#### Não Funciona ####
# # Acessar fast_info
info = ticker_data.info

In [6]:
yahoo_symbol_recommendations = ticker_data.recommendations
# yahoo_symbol_recommendations

In [8]:
# Dividendos
cashflow = ticker_data.cashflow
# cashflow

In [9]:
# Notícias
balance_sheet = ticker_data.balance_sheet
# balance_sheet

In [10]:
yahoo_symbol_sustainability = ticker_data.sustainability
# yahoo_symbol_sustainability

In [9]:
# Eventos corporativos
calendar = ticker_data.calendar
# calendar

In [10]:
# # Datas disponíveis para opções
# options_dates = ticker_data.options
# print("Datas de opções:", options_dates)

# # Obter opções para uma data específica
# options = ticker_data.option_chain(options_dates[0])
# print("Opções de compra (calls):", options.calls)
# print("Opções de venda (puts):", options.puts)

In [11]:
# # Demonstrativo financeiro
# financials = ticker_data.financials
# print("Demonstrativo financeiro:", financials)

# # Balanço patrimonial
# balance_sheet = ticker_data.balance_sheet
# print("Balanço patrimonial:", balance_sheet)

# # Fluxo de caixa
# cashflow = ticker_data.cashflow
# print("Fluxo de caixa:", cashflow)

Informações financeiras mais detalhadas da empresa.

In [12]:
# # Dados de sustentabilidade
# sustainability = ticker_data.sustainability
# print(sustainability)

Informações de ESG (Environmental, Social, Governance).

In [13]:
# # Ações em circulação
# shares = ticker_data.shares
# print("Ações em circulação:", shares)

Obtenha informações sobre ações em circulação e histórico de splits.

In [14]:
# Principais investidores institucionais
inst_holders = ticker_data.institutional_holders
# inst_holders


Dados sobre investidores institucionais e sua participação.

In [15]:
# Principais investidores institucionais
holders = ticker_data.major_holders
# holders


In [23]:
def get_yahoo_data_history(symbol : str, period : str, interval: str, start = '1900-01-01', end = datetime.now(), prepost : bool = True):
        '''
        Data collection from yahoo

        Parameters:
        period : str
            Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
        interval : str
            Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo Intraday data cannot extend last 60 days
        start: str
            Download start date string (YYYY-MM-DD) or _datetime, inclusive. Default is 1900-01-01 E.g. for start="2020-01-01", the first data point will be on "2020-01-01"
        end: str
            Download end date string (YYYY-MM-DD) or _datetime, exclusive. Default is now E.g. for end="2023-01-01", the last data point will be on "2022-12-31"
        prepost : bool
            Include Pre and Post market data in results? Default is False
        debug: bool
            If passed as False, will suppress error message printing to console.
        '''
        yahoo_data_history = yf.Ticker(symbol).history(period=period, interval=interval, start=start, end=end, prepost=prepost)
        return yahoo_data_history

history = get_yahoo_data_history(symbol="MSFT",period='1y', interval='1d', end = datetime.now(), prepost=True)

In [24]:
import talib
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed


class ParamsOptimization:
    """
    A class for optimizing technical indicators' parameters and evaluating strategy performance.
    """

    def __init__(self, ticker, timeframe):
        self.ticker = ticker
        self.timeframe = timeframe
        self.best_params = pd.DataFrame()


    def optimize_crossover(self, data, ticker):
        """
        Optimize EMA crossover strategy.
        """
        ema1_periods = [10, 15, 20]
        ema2_periods = [25, 30, 50]
        ema3_periods = [100, 150, 200]

        combinations = list(itertools.product(ema1_periods, ema2_periods, ema3_periods))

        # Parallel execution for speed
        results = Parallel(n_jobs=-1)(delayed(self.simulate_crossover)(
            data, ticker, l1, l2, l3) for l1, l2, l3 in tqdm(combinations, desc="Optimizing EMA Crossover"))

        # Combine results into a DataFrame
        results_df = pd.DataFrame(results)
        best_result = results_df.loc[results_df['Expectancy'].idxmax()]

        return best_result

    def simulate_crossover(self, data, ticker, l1, l2, l3):
        """
        Simulate crossover strategy and calculate metrics.
        """
        # Calculate EMAs
        data['ema1'] = talib.EMA(data['close'], timeperiod=l1)
        data['ema2'] = talib.EMA(data['close'], timeperiod=l2)
        data['ema3'] = talib.EMA(data['close'], timeperiod=l3)

        # Generate signals
        data['signal'] = np.where((data['ema1'] > data['ema2']) & (data['ema2'] > data['ema3']), 1,
                                  np.where((data['ema1'] < data['ema2']) & (data['ema2'] < data['ema3']), -1, 0))
        data['returns'] = data['close'].pct_change() * data['signal'].shift(1)

        # Calculate metrics
        sharpe = self.calculate_sharpe(data['returns'])
        max_drawdown = self.calculate_max_drawdown(data['returns'])
        expectancy = self.calculate_expectancy(data['returns'])

        return {
            'Ticker': ticker,
            'Best_EMA1': l1,
            'Best_EMA2': l2,
            'Best_EMA3': l3,
            'Sharpe': sharpe,
            'Max_Drawdown': max_drawdown,
            'Expectancy': expectancy
        }

    def optimize_bbands(self, data, ticker):
        """
        Optimize Bollinger Bands strategy.
        """
        sma_periods = [10, 15, 20]
        std_devs = [1, 2, 3]

        combinations = list(itertools.product(sma_periods, std_devs))

        # Parallel execution for speed
        results = Parallel(n_jobs=-1)(delayed(self.simulate_bbands)(
            data, ticker, period, std) for period, std in tqdm(combinations, desc="Optimizing Bollinger Bands"))

        # Combine results into a DataFrame
        results_df = pd.DataFrame(results)
        best_result = results_df.loc[results_df['Expectancy'].idxmax()]

        return best_result

    def simulate_bbands(self, data, ticker, period, std):
        """
        Simulate Bollinger Bands strategy and calculate metrics.
        """
        # Calculate Bollinger Bands
        upperband, middleband, lowerband = talib.BBANDS(
            data['close'], timeperiod=period, nbdevup=std, nbdevdn=std, matype=0
        )

        # Generate signals
        data['signal'] = np.where(data['close'] < lowerband, 1,
                                  np.where(data['close'] > upperband, -1, 0))
        data['returns'] = data['close'].pct_change() * data['signal'].shift(1)

        # Calculate metrics
        sharpe = self.calculate_sharpe(data['returns'])
        max_drawdown = self.calculate_max_drawdown(data['returns'])
        expectancy = self.calculate_expectancy(data['returns'])

        return {
            'Ticker': ticker,
            'Best_Period': period,
            'Best_Std': std,
            'Sharpe': sharpe,
            'Max_Drawdown': max_drawdown,
            'Expectancy': expectancy
        }

    @staticmethod
    def calculate_sharpe(returns, risk_free_rate=0.025):
        """
        Calculate Sharpe Ratio.
        """
        mean_return = returns.mean()
        std_dev = returns.std()
        if std_dev == 0:
            return 0
        return (mean_return - risk_free_rate) / std_dev

    @staticmethod
    def calculate_max_drawdown(returns):
        """
        Calculate Max Drawdown.
        """
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.cummax()
        drawdown = running_max - cumulative
        return drawdown.max()

    @staticmethod
    def calculate_expectancy(returns):
        """
        Calculate Expectancy.
        """
        wins = returns[returns > 0]
        losses = returns[returns < 0]
        win_rate = len(wins) / len(returns) if len(returns) > 0 else 0
        loss_rate = 1 - win_rate
        avg_win = wins.mean() if len(wins) > 0 else 0
        avg_loss = losses.mean() if len(losses) > 0 else 0
        return (win_rate * avg_win) - (loss_rate * avg_loss)



# # Instanciar e otimizar
# optimizer = ParamsOptimization(ticker=["AAPL", "GOOG"], market_type="stocks", timeframe="1d")
# best_params = optimizer.optimize()

# # Exibir resultados
# print(best_params)

ModuleNotFoundError: No module named 'talib'